# DIVERSE Model - Training Analysis

Detaylı training analizi ve grafikler

In [ ]:
# Setup
from google.colab import drive, files
drive.mount('/content/drive')

import json
import matplotlib.pyplot as plt
import numpy as np
import glob

print("✓ Kütüphaneler yüklendi")

In [ ]:
# Log'ları yükle
log_dir = '/content/drive/MyDrive/lora_checkpoints/diverse/logs'
log_files = glob.glob(f"{log_dir}/training_log_*.jsonl")

if not log_files:
    print("❌ Log dosyası bulunamadı!")
else:
    log_file = log_files[0]
    print(f"✓ Log dosyası: {log_file}")
    
    with open(log_file, 'r') as f:
        logs = [json.loads(line) for line in f]
    
    # Ayır
    train_logs = [log for log in logs if 'train_loss' in log]
    eval_logs = [log for log in logs if 'eval_loss' in log]
    
    print(f"✓ Train logs: {len(train_logs)}")
    print(f"✓ Eval logs: {len(eval_logs)}")

In [ ]:
# İstatistikler
train_steps = [log['step'] for log in train_logs]
train_losses = [log['train_loss'] for log in train_logs]
learning_rates = [log.get('learning_rate', 0) for log in train_logs]

print("📊 DIVERSE Model İstatistikleri")
print("=" * 50)
print(f"Toplam Step: {len(train_logs)}")
print(f"İlk Train Loss: {train_losses[0]:.4f}")
print(f"Son Train Loss: {train_losses[-1]:.4f}")
print(f"İyileşme: {((train_losses[0] - train_losses[-1]) / train_losses[0] * 100):.1f}%")
print(f"Min Loss: {min(train_losses):.4f}")
print(f"Max Loss: {max(train_losses):.4f}")

if eval_logs:
    eval_losses = [log['eval_loss'] for log in eval_logs]
    print(f"\nEn İyi Eval Loss: {min(eval_losses):.4f}")
    best_idx = eval_losses.index(min(eval_losses))
    print(f"En İyi Checkpoint: step-{eval_logs[best_idx]['step']}")

In [ ]:
# Grafik 1: Training Loss
plt.figure(figsize=(12, 6))
plt.plot(train_steps, train_losses, '#10AC84', linewidth=2.5, alpha=0.8, label='Train Loss')
plt.xlabel('Step', fontsize=14, fontweight='bold')
plt.ylabel('Loss', fontsize=14, fontweight='bold')
plt.title('DIVERSE Model - Training Loss Over Time', fontsize=16, fontweight='bold')
plt.grid(True, alpha=0.3, linestyle='--')
plt.legend(fontsize=12)
plt.tight_layout()
plt.savefig('diverse_train_loss.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print("✓ Grafik 1 kaydedildi: diverse_train_loss.png")

In [ ]:
# Grafik 2: Validation Loss
if eval_logs:
    eval_steps = [log['step'] for log in eval_logs]
    eval_losses = [log['eval_loss'] for log in eval_logs]
    
    plt.figure(figsize=(12, 6))
    plt.plot(eval_steps, eval_losses, '#F79F1F', linewidth=2.5, marker='o', markersize=10, label='Validation Loss')
    plt.xlabel('Step', fontsize=14, fontweight='bold')
    plt.ylabel('Loss', fontsize=14, fontweight='bold')
    plt.title('DIVERSE Model - Validation Loss', fontsize=16, fontweight='bold')
    plt.grid(True, alpha=0.3, linestyle='--')
    plt.legend(fontsize=12)
    plt.tight_layout()
    plt.savefig('diverse_eval_loss.png', dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()
    print("✓ Grafik 2 kaydedildi: diverse_eval_loss.png")

In [ ]:
# Grafik 3: Learning Rate Schedule
if any(learning_rates):
    plt.figure(figsize=(12, 6))
    plt.plot(train_steps, learning_rates, '#9B59B6', linewidth=2.5, label='Learning Rate')
    plt.xlabel('Step', fontsize=14, fontweight='bold')
    plt.ylabel('Learning Rate', fontsize=14, fontweight='bold')
    plt.title('DIVERSE Model - Learning Rate Schedule', fontsize=16, fontweight='bold')
    plt.grid(True, alpha=0.3, linestyle='--')
    plt.legend(fontsize=12)
    plt.ticklabel_format(style='scientific', axis='y', scilimits=(0,0))
    plt.tight_layout()
    plt.savefig('diverse_learning_rate.png', dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()
    print("✓ Grafik 3 kaydedildi: diverse_learning_rate.png")

In [ ]:
# Grafik 4: Complete Analysis (4-in-1)
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Train Loss
axes[0, 0].plot(train_steps, train_losses, '#10AC84', linewidth=2.5)
axes[0, 0].set_xlabel('Step', fontweight='bold')
axes[0, 0].set_ylabel('Loss', fontweight='bold')
axes[0, 0].set_title('Training Loss', fontweight='bold', fontsize=14)
axes[0, 0].grid(True, alpha=0.3)

# Eval Loss
if eval_logs:
    axes[0, 1].plot(eval_steps, eval_losses, '#F79F1F', linewidth=2.5, marker='o', markersize=8)
    axes[0, 1].set_xlabel('Step', fontweight='bold')
    axes[0, 1].set_ylabel('Loss', fontweight='bold')
    axes[0, 1].set_title('Validation Loss', fontweight='bold', fontsize=14)
    axes[0, 1].grid(True, alpha=0.3)

# Learning Rate
if any(learning_rates):
    axes[1, 0].plot(train_steps, learning_rates, '#9B59B6', linewidth=2.5)
    axes[1, 0].set_xlabel('Step', fontweight='bold')
    axes[1, 0].set_ylabel('Learning Rate', fontweight='bold')
    axes[1, 0].set_title('Learning Rate Schedule', fontweight='bold', fontsize=14)
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].ticklabel_format(style='scientific', axis='y', scilimits=(0,0))

# Smoothed Loss
window = 10
smoothed = np.convolve(train_losses, np.ones(window)/window, mode='valid')
axes[1, 1].plot(train_steps[:len(smoothed)], smoothed, '#E74C3C', linewidth=2.5)
axes[1, 1].set_xlabel('Step', fontweight='bold')
axes[1, 1].set_ylabel('Loss (Smoothed)', fontweight='bold')
axes[1, 1].set_title('Smoothed Training Loss', fontweight='bold', fontsize=14)
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('DIVERSE Model - Complete Training Analysis', fontsize=18, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig('diverse_complete_analysis.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print("✓ Grafik 4 kaydedildi: diverse_complete_analysis.png")

In [ ]:
# Grafikleri indir
print("\nGrafikleri indiriyor...")
files.download('diverse_train_loss.png')
files.download('diverse_eval_loss.png')
files.download('diverse_learning_rate.png')
files.download('diverse_complete_analysis.png')
print("\n✓ Tüm DIVERSE grafikleri indirildi!")